# Assumptions & Procedure
For the sake of simplicity & modularity, I have created the code for the chatbot such that it searches for products from the amazon_data that i have already scraped in task 1.



It works on the data provided in the dataset and answers any queries with relevant info from the data.

In the future, it can be modified to work on any webpage as an extension to the website, instead of using already scraped data.

# Chatbot code

In [10]:
import pandas as pd
import spacy

# Load the CSV file into a DataFrame
df = pd.read_csv('amazon_data.csv')

def search_product(product_name):
    # Search for the product in the DataFrame
    product_data = df[df['title'].str.contains(product_name, case=False, na=False)]
    if not product_data.empty:
        return product_data
    else:
        return None

def chatbot():
    print("Hello, this is SHopy, your shopping assistant :) What do you want to chat about?")

    while True:
        product_name = input("Enter the name of the product you want to inquire about or type 'exit' to end: ")
        if product_name.lower() == 'exit':
            break

        product_data = search_product(product_name)
        if product_data is None:
            print("I'm sorry, I couldn't find that product.")
            continue

        # If more than one product matches, print them all and ask the user to choose one
        if len(product_data) > 1:
            print("I found multiple products that match your query. Here they are:")
            for i, (index, product) in enumerate(product_data.iterrows(), start=1):
                print(f"{i}: {product['title']}")
            chosen_index = int(input("Please enter the number of the product you want to inquire about: ")) - 1
            product_data = product_data.iloc[chosen_index]

        nlp = spacy.load('en_core_web_md')  # make sure you have this model downloaded
        queries = {
            'title': ['name', 'title', 'brand', 'product name', 'what is it called'],
            'price': ['price', 'cost', 'how much', 'value', 'worth', 'expense'],
            'description': ['details', 'detail', 'specs', 'specifications', 'features'],
            'rating': ['rating','ratings','rated', 'stars', 'feedback', 'reviews'],
            'reviews_count': ['reviews count','no of reviews', 'number of reviews', 'how many reviews'],
            'availability': ['deliver','delivery','available', 'availability', 'in stock', 'can I buy it'],
            'all_info': ['display all data',
                         'display all info',
                         'display all information',
                         'give all info',
                         'show everything',
                         'show all details']
        }

        while True:
            statement = input("Enter your question about the product or type 'new' for a new product or type 'exit' to end: ")
            if statement.lower() == 'exit':
                return
            elif statement.lower() == 'new':
                break

            statement_doc = nlp(statement)
            found_match = False

            # Check if the user's statement contains any of the keywords
            for key, synonyms in queries.items():
                for synonym in synonyms:
                    if synonym in statement.lower():
                        if key == 'all_info':
                            print(f"Here are all the details of the product:\n{product_data}")
                        else:
                            print(f"The {key} of the product is: {str(product_data[key])}")
                        found_match = True
                        break

                if found_match:
                    break

            # If no keyword matches, check for specific details in the product description
            details = str(product_data.get('description', ''))

            for sentence in details.split('|'):  # split on '|' instead of '.'
                sentence = sentence.strip()  # remove leading and trailing whitespace and punctuation
                for token in statement_doc:
                    # Ignore stop words and punctuation
                    if not token.is_stop and not token.is_punct and token.text.lower() in sentence.lower():
                        print(f"I found this information about your query:\n {sentence}")
                        found_match = True
                        break

                if found_match:
                    break

            if not found_match:
                print("I'm sorry, I didn't understand your question. Could you please rephrase it?")

# Example usage:
chatbot()




Hello, this is SHopy, your shopping assistant :) What do you want to chat about?
Enter the name of the product you want to inquire about or type 'exit' to end: Samsung
I found multiple products that match your query. Here they are:
1: Samsung Galaxy M04 Light Green, 4GB RAM, 64GB Storage | Upto 8GB RAM with RAM Plus | MediaTek Helio P35 Octa-core Processor | 5000 mAh Battery | 13MP Dual Camera
2: Samsung Galaxy M13 (Stardust Brown, 4GB, 64GB Storage) | 6000mAh Battery | Upto 8GB RAM with RAM Plus
3: Samsung Galaxy M13 (Midnight Blue, 4GB, 64GB Storage) | 6000mAh Battery | Upto 8GB RAM with RAM Plus
Please enter the number of the product you want to inquire about: 3
Enter your question about the product or type 'new' for a new product or type 'exit' to end: Is it rated good
The rating of the product is: 4.1 out of 5 stars
Enter your question about the product or type 'new' for a new product or type 'exit' to end: WHat kind of camera does it have
I found this information about your query